In [1]:
import sys
from pathlib import Path

In [2]:
module_path = Path.cwd().parent.parent
if module_path not in sys.path:
    sys.path.insert(0, str(module_path))

In [3]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from sklearn.model_selection import train_test_split


In [4]:
from src.loader import TextLoader

In [5]:
!mkdir -p data/

In [6]:
loader = TextLoader()

In [10]:
for text in loader.iterate(verbose=True, purify_text=True, purify_discourses=True):
    if text.id in ("F91D7BB4277C", "354946A1CA46"): # Broken, to delete
        continue

    words = [word.rstrip(".,?") for word in text.words]
    prev_end = 0
    for disc in text.discourses[2:3]:
        start = text.text.index(disc.text)
        # new_idx = (start, start + len(disc.text))

        disc_words = disc.text.split()
        if (first_word := disc_words[0].rstrip(".,?")) not in words:
            fixed = first_word
            curr_ind = start - 1
            while curr_ind >= 0 and (curr_char := text[curr_ind]) != " ":
                curr_ind -= 1
                fixed = curr_char + fixed

            print(f"\nText: {text.id}")
            print(f"`{first_word}` could not be found, but it's probably: `{fixed}`")

        # end = start + len(disc_words)
        # prev_end = end

        # text_slice = " ".join(words[start:end]).rstrip(".")

        # print(start, end)
        # print(" ".join(words[start:end]))
        # print()
        # print(disc.text)
        # print("----")



2664 / 15594
Text: 4A4ED75E7112
`ou` could not be found, but it's probably: `You`
3667 / 15594
Text: 32E2949B0C66
`hen` could not be found, but it's probably: `when`
4007 / 15594
Text: 280D13EB16F0
`s` could not be found, but it's probably: `is`
6145 / 15594
Text: 8BBFBE748A23
`f` could not be found, but it's probably: `if`
6654 / 15594
Text: 13E32905929D
`his` could not be found, but it's probably: `This`
6979 / 15594
Text: 0A23334002E0
`hey` could not be found, but it's probably: `they`
7244 / 15594
Text: CE4EFA6441C4
`he` could not be found, but it's probably: `the`
7460 / 15594
Text: 1C58F2AF08B4
`t` could not be found, but it's probably: `It`
10396 / 15594
Text: 671D0569C835
`his` could not be found, but it's probably: `this`
10401 / 15594
Text: 9FDB425F1FE9
`t` could not be found, but it's probably: `it`
10550 / 15594
Text: 4A8FCC8C7D9B
`hat` could not be found, but it's probably: `that`
10785 / 15594
Text: B689C28463CB
`ike` could not be found, but it's probably: `like`
11070 / 

In [7]:
text = loader.load_text_with_id("63687EA6EB28", purify_text=True, purify_discourses=True)

In [8]:
for disc in text.discourses:
    print(disc, disc.text in text.text)

--- 1618061680644 (0 -> 61 | 0 -> 11) - Lead ---
Many people that think the face of mars was created by aliens
------------------------------------------------ True
--- 1618061692866 (66 -> 96 | 13 -> 18) - Position ---
it was really just a landform
------------------------------------------------------ True
--- 1618061731020 (109 -> 175 | 22 -> 33) - Claim ---
scientits say its just like a land formation that happens on earth
----------------------------------------------------- True
--- 1618061756973 (176 -> 333 | 33 -> 60) - Evidence ---
this land formation are more common around american west the article says it reminds me mist of middle butte in the snake river plain of idaho says garvin
-------------------------------------------------------- True
--- 1618061802760 (334 -> 441 | 61 -> 74) - Claim ---
then garvine said Malins team captured an extrordinary photousing the cameras absolute maximum resolution
----------------------------------------------------- True
--- 1618061841006

In [9]:
print(text)

Many people that think the face of mars was created by aliens but it was really just a landform here is why. scientits say its just like a land formation that happens on earth this land formation are more common around american west the article says it reminds me mist of middle butte in the snake river plain of idaho says garvin. then garvine said Malins team captured an extrordinary photousing the cameras absolute maximum resolution each pixel in 2001 image spans 1. 56 meters compared to 43 meters per pixel in the best 1976 viking photo in April 5 1998 MOC or mars orbiter camera team snapped a picture ten times shapper only for the thousands of anxious web surfers to see a natural landform on mars and not any aliens. but yet not everyone was satisfied on April 98 a cloudy time of year on the red planet the camera on boared MGS had to peer through the wispy clouds to see the face perhaps skeptis say alien markings were hidden by the haze. now many people dont talk about this topic alot

In [ ]:
print(text)

In [ ]:
st, en = text.discourses[1].ind_start, text.discourses[1].ind_end
print(text.text[st:en])

In [ ]:
p_st, p_en = text.discourses[1].predictionstring[0], text.discourses[1].predictionstring[-1]
print(text.words[p_st : p_en + 1])


In [ ]:
# text = loader.load_text_with_id("E881FAAEC690")
# text = loader.load_text_with_id("6FD9A4641AD7")
for text in loader.iterate(verbose=True):
    ...

In [ ]:
text.id

In [ ]:
for disc in text.discourses:
    print(disc)


In [ ]:
print(text)

In [ ]:
def create_doc(
    doc_type: str, offset: int = 0, limit: int = 0, shuffle: bool = True, seed: int = 8888
) -> None:
    current_ends = []
    output = []
    DS_count = 0
    DE_count = 0
    for text_no, text in enumerate(
        loader.iterate(offset=offset, limit=limit, shuffle=shuffle, seed=seed)
    ):
        print(f"\r{text_no + 1:3} / {limit}", end="")
        current_ends = [
            (disc.predictionstring[0], disc.predictionstring[-1]) for disc in text.discourses
        ]

        curr_start, curr_end = current_ends.pop(0)
        for word_ind, word in enumerate(text.words):
            if word_ind > curr_end:
                if not current_ends:
                    break

                curr_start, curr_end = current_ends.pop(0)

            if word_ind == curr_start:
                output.append(f"{word} DS\n")
                DS_count += 1
            elif word_ind == curr_end:
                output.append(f"{word} DE\n")
                DE_count += 1
            else:
                output.append(f"{word} O\n")

            if DE_count > DS_count:
                raise Exception(f"Wut for {word_ind}")

            if "." in word:
                output.append("\n")

        output.append("<DOC>\n")

    output = output[:-1]  # remove last <DOC>

    with open(f"data/NER_{doc_type}.txt", "w") as f:
        f.writelines(output)


In [ ]:
text = loader.load_text_with_id("DBF7EB6A9E02")
disc, = [disc for disc in text.discourses if disc.id == 1622489430075]
print(disc.text)
print(text[disc.ind_start:disc.ind_end].split())
pred_start, pred_end = int(disc.predictionstring[0]), int(disc.predictionstring[-1])
print(text.words[pred_start:pred_end+1]) # TODO: Problem with predictionstring

In [ ]:
text = loader.load_text_with_id("DBF7EB6A9E02")
for disc in text.discourses:
    print(disc.id)
    char_start = disc.ind_start
    char_end = disc.ind_end
    word_start = len(text[:char_start].split())
    word_end = word_start + len(text[char_start:char_end].split())
    word_end = min(word_end, len(text.split()))
    print((word_start, word_end))
    
    d_text = disc.text.split()
    e_text = text.words[word_start:word_end]
    if d_text[0] != e_text[0]:
        d_text.remove(d_text[0])
        e_text.remove(e_text[-1])

    for ind, d in enumerate(d_text):
        print(f"{d} == {e_text[ind]} --> {d == e_text[ind]}")

In [ ]:
current_ends = [
    (disc.predictionstring[0], disc.predictionstring[-1]) for disc in text.discourses
]
current_ends


In [ ]:
curr_start, curr_end = current_ends.pop(0)
for word_ind, word in enumerate(text.words):
    if word_ind > curr_end:
        if not current_ends:
            break

        curr_start, curr_end = current_ends.pop(0)

    if word_ind == curr_start:
        print(f"{word} ({word_ind}) DS")
    elif word_ind == curr_end:
        print(f"{word} ({word_ind}) DE")
    else:
        print(f"{word} ({word_ind}) O")


In [ ]:
len(loader)


In [ ]:
# train_size = 0.8
# dev_size = 0.15
# test_size = 0.05

train_size, dev_size = train_test_split(
    range(len(loader)), test_size=0.2, random_state=8888
)
dev_size, test_size = train_test_split(
    dev_size, test_size=0.25, random_state=8888
)
train_size, test_size, dev_size = len(train_size), len(test_size), len(dev_size)

print(f"Train size: {train_size}, Dev size: {dev_size}, Test size: {test_size}")


In [ ]:
create_doc("train", offset=0, limit=train_size, shuffle=False, seed=8888)

In [ ]:
create_doc("dev", offset=train_size, limit=dev_size)

In [ ]:
create_doc("test", offset=train_size + dev_size, limit=test_size)